In [3]:
import polars as pl

FILE_PATH = "guillaume.txt" # Ton fichier

# Liste des colonnes numériques à nettoyer (virgule -> point -> float)
# Note : On inclut tout ce qui est chiffré
FLOAT_COLS = [
    "Montant", "TauxImpNb_RB", "TauxImpNB_CPM", 
    "ScoringFP1", "ScoringFP2", "ScoringFP3",
    "DiffDateTr1", "DiffDateTr2", "DiffDateTr3",
    "CA3TRetMtt", "CA3TR", "EcartNumCheq", 
    "NbrMagasin3J", "D2CB"
]

# Liste des colonnes qui sont des entiers simples (sans virgule à la base)
# On devra aussi les caster après le nettoyage
INT_COLS = ["FlagImpaye", "CodeDecision", "VerifianceCPT1", "VerifianceCPT2", "VerifianceCPT3"]

q = (
    pl.scan_csv(
        FILE_PATH, 
        separator=";", 
        # L'ASTUCE ULTIME : On lit tout en String (texte) par défaut.
        # Comme ça, Polars ne crashe pas en voyant "IDAvis..." au milieu des chiffres.
        infer_schema_length=0 
    )
    # 1. On filtre la ligne de header qui traîne au milieu
    # On garde seulement les lignes où ZIBZIN n'est pas écrit "ZIBZIN"
    .filter(pl.col("ZIBZIN") != "ZIBZIN")
    
    # 2. Maintenant que les intrus sont partis, on nettoie et on type
    .with_columns([
        # Pour les floats : replace ',' -> '.' -> cast
        pl.col(col).str.replace(",", ".").cast(pl.Float64) 
        for col in FLOAT_COLS
    ])
    .with_columns([
        # Pour les entiers : cast direct (maintenant que le texte est filtré)
        pl.col(col).cast(pl.Int64) 
        for col in INT_COLS
    ])
    .with_columns(
        # Date : String -> Datetime
        pl.col("DateTransaction").str.to_datetime("%Y-%m-%d %H:%M:%S")
    )
)

print("Plan de chargement 'Dirty CSV' prêt.")

Plan de chargement 'Dirty CSV' prêt.


In [4]:
try:
    df = q.collect()
    print(f"✅ Succès ! Dimensions : {df.shape}")
    print("Preuve que c'est propre (types) :")
    print(df.schema)
except Exception as e:
    print(f"❌ Erreur : {e}")

✅ Succès ! Dimensions : (4646773, 23)
Preuve que c'est propre (types) :
Schema({'ZIBZIN': String, 'IDAvisAutorisationCheque': String, 'FlagImpaye': Int64, 'Montant': Float64, 'DateTransaction': Datetime(time_unit='us', time_zone=None), 'CodeDecision': Int64, 'VerifianceCPT1': Int64, 'VerifianceCPT2': Int64, 'VerifianceCPT3': Int64, 'D2CB': Float64, 'ScoringFP1': Float64, 'ScoringFP2': Float64, 'ScoringFP3': Float64, 'TauxImpNb_RB': Float64, 'TauxImpNB_CPM': Float64, 'EcartNumCheq': Float64, 'NbrMagasin3J': Float64, 'DiffDateTr1': Float64, 'DiffDateTr2': Float64, 'DiffDateTr3': Float64, 'CA3TRetMtt': Float64, 'CA3TR': Float64, 'Heure': String})


In [5]:
# Affiche les 5 premières lignes (Comme Pandas)
print(df.head(5))

# Affiche la structure, les types, et un aperçu des valeurs (Top pour vérifier le parsing)
print("\n--- Structure et Types ---")
df.glimpse()

shape: (5, 23)
┌──────────────┬─────────────┬────────────┬─────────┬───┬─────────────┬────────────┬───────┬───────┐
│ ZIBZIN       ┆ IDAvisAutor ┆ FlagImpaye ┆ Montant ┆ … ┆ DiffDateTr3 ┆ CA3TRetMtt ┆ CA3TR ┆ Heure │
│ ---          ┆ isationCheq ┆ ---        ┆ ---     ┆   ┆ ---         ┆ ---        ┆ ---   ┆ ---   │
│ str          ┆ ue          ┆ i64        ┆ f64     ┆   ┆ f64         ┆ f64        ┆ f64   ┆ str   │
│              ┆ ---         ┆            ┆         ┆   ┆             ┆            ┆       ┆       │
│              ┆ str         ┆            ┆         ┆   ┆             ┆            ┆       ┆       │
╞══════════════╪═════════════╪════════════╪═════════╪═══╪═════════════╪════════════╪═══════╪═══════╡
│ A01301000490 ┆ 78643044    ┆ 0          ┆ 20.0    ┆ … ┆ 4.0         ┆ 20.0       ┆ 0.0   ┆ 27134 │
│ 812670306093 ┆             ┆            ┆         ┆   ┆             ┆            ┆       ┆       │
│ 1            ┆             ┆            ┆         ┆   ┆             ┆     

In [6]:
# --- Feature Engineering & Nettoyage ---

# 1. On extrait l'heure (0-23) depuis la DateTransaction
#    C'est plus pertinent que des secondes brutes pour un arbre de décision.
df_clean = df.with_columns(
    pl.col("DateTransaction").dt.hour().alias("HourOfDay")
)

# 2. On supprime les colonnes inutiles pour la prédiction
#    - ZIBZIN : Identifiant client (trop unique, risque d'overfitting)
#    - IDAvis... : Identifiant transaction (inutile)
#    - Heure : Redondant avec DateTransaction
#    - CodeDecision : C'est de la triche (data leakage) ! Le sujet dit de l'enlever.
cols_to_drop = ["ZIBZIN", "IDAvisAutorisationCheque", "Heure", "CodeDecision"]

# On ne supprime que celles qui existent (au cas où tu relances la cellule)
existing_cols_to_drop = [c for c in cols_to_drop if c in df_clean.columns]
df_clean = df_clean.drop(existing_cols_to_drop)

print("--- Dataset Nettoyé ---")
print(f"Dimensions : {df_clean.shape}")
print("Nouvelles colonnes :")
print(df_clean.columns)

# Petit check sur la nouvelle colonne Heure
print("\n--- Distribution des transactions par heure ---")
print(df_clean["HourOfDay"].value_counts(sort=True).head())

--- Dataset Nettoyé ---
Dimensions : (4646773, 20)
Nouvelles colonnes :
['FlagImpaye', 'Montant', 'DateTransaction', 'VerifianceCPT1', 'VerifianceCPT2', 'VerifianceCPT3', 'D2CB', 'ScoringFP1', 'ScoringFP2', 'ScoringFP3', 'TauxImpNb_RB', 'TauxImpNB_CPM', 'EcartNumCheq', 'NbrMagasin3J', 'DiffDateTr1', 'DiffDateTr2', 'DiffDateTr3', 'CA3TRetMtt', 'CA3TR', 'HourOfDay']

--- Distribution des transactions par heure ---
shape: (5, 2)
┌───────────┬────────┐
│ HourOfDay ┆ count  │
│ ---       ┆ ---    │
│ i8        ┆ u32    │
╞═══════════╪════════╡
│ 11        ┆ 534742 │
│ 17        ┆ 497154 │
│ 10        ┆ 492316 │
│ 18        ┆ 471816 │
│ 16        ┆ 466012 │
└───────────┴────────┘


In [7]:
# On compte les nulls pour chaque colonne
null_counts = df.null_count()

# Affichage transposé pour lire facilement si tout est à 0
print("--- Valeurs manquantes par colonne ---")
print(null_counts.transpose(include_header=True, header_name="Colonne", column_names=["Nb_Nulls"]))

# Si tu veux voir uniquement s'il y a un problème quelque part :
total_nulls = df.null_count().sum_horizontal().sum()
print(f"\nNombre total de valeurs manquantes dans tout le dataset : {total_nulls}")

--- Valeurs manquantes par colonne ---
shape: (23, 2)
┌──────────────────────────┬──────────┐
│ Colonne                  ┆ Nb_Nulls │
│ ---                      ┆ ---      │
│ str                      ┆ u32      │
╞══════════════════════════╪══════════╡
│ ZIBZIN                   ┆ 0        │
│ IDAvisAutorisationCheque ┆ 0        │
│ FlagImpaye               ┆ 0        │
│ Montant                  ┆ 0        │
│ DateTransaction          ┆ 0        │
│ …                        ┆ …        │
│ DiffDateTr2              ┆ 0        │
│ DiffDateTr3              ┆ 0        │
│ CA3TRetMtt               ┆ 0        │
│ CA3TR                    ┆ 0        │
│ Heure                    ┆ 0        │
└──────────────────────────┴──────────┘

Nombre total de valeurs manquantes dans tout le dataset : 0


In [8]:
# On sélectionne les colonnes numériques (Float)
# Et on utilise le describe de Polars qui est très lisible
print("--- Statistiques des variables numériques ---")
print(df.select(pl.col(pl.Float64)).describe())

--- Statistiques des variables numériques ---
shape: (9, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆ Montant   ┆ D2CB      ┆ ScoringFP ┆ … ┆ DiffDateT ┆ DiffDateT ┆ CA3TRetMt ┆ CA3TR    │
│ ---       ┆ ---       ┆ ---       ┆ 1         ┆   ┆ r2        ┆ r3        ┆ t         ┆ ---      │
│ str       ┆ f64       ┆ f64       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ f64      │
│           ┆           ┆           ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 4.646773e ┆ 4.646773e ┆ 4.646773e ┆ … ┆ 4.646773e ┆ 4.646773e ┆ 4.646773e ┆ 4.646773 │
│           ┆ 6         ┆ 6         ┆ 6         ┆   ┆ 6         ┆ 6         ┆ 6         ┆ e6       │
│ null_coun ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 0.0      │
│ t         ┆           ┆     

In [9]:
# On compte les 0 (légitime) et les 1 (fraude)
imbalance = df["FlagImpaye"].value_counts(sort=True)

# On ajoute une colonne de pourcentage pour y voir clair
imbalance = imbalance.with_columns(
    (pl.col("count") / df.height * 100).alias("Pourcentage (%)")
)

print("--- Répartition des Classes (0 vs 1) ---")
print(imbalance)

--- Répartition des Classes (0 vs 1) ---
shape: (2, 3)
┌────────────┬─────────┬─────────────────┐
│ FlagImpaye ┆ count   ┆ Pourcentage (%) │
│ ---        ┆ ---     ┆ ---             │
│ i64        ┆ u32     ┆ f64             │
╞════════════╪═════════╪═════════════════╡
│ 0          ┆ 4616778 ┆ 99.354498       │
│ 1          ┆ 29995   ┆ 0.645502        │
└────────────┴─────────┴─────────────────┘


In [11]:
# 1. Feature Engineering: Extract Hour from Date
df_clean = df.with_columns(
    pl.col("DateTransaction").dt.hour().alias("HourOfDay")
)

# 2. Cleanup: Drop identifiers and leakage columns
# CodeDecision is explicitly forbidden by the subject (leakage)
cols_to_drop = [
    "ZIBZIN", 
    "IDAvisAutorisationCheque", 
    "Heure", 
    "CodeDecision"
]
# Filter to ensure we only drop columns that actually exist
existing_cols = [c for c in cols_to_drop if c in df_clean.columns]
df_clean = df_clean.drop(existing_cols)

print(f"Dataset cleaned. New shape: {df_clean.shape}")

Dataset cleaned. New shape: (4646773, 20)


In [12]:
from datetime import datetime

# Define split date: September 1st, 2017
split_date = datetime(2017, 9, 1)

# Polars filter
train_df = df_clean.filter(pl.col("DateTransaction") < split_date)
test_df = df_clean.filter(pl.col("DateTransaction") >= split_date)

print(f"Train set: {train_df.shape[0]} samples")
print(f"Test set : {test_df.shape[0]} samples")

Train set: 3899362 samples
Test set : 747411 samples


In [13]:
import os

# Création d'un dossier 'data/processed' s'il n'existe pas
os.makedirs("data/processed", exist_ok=True)

# Sauvegarde ultra-rapide avec Polars (format Parquet)
# On sauvegarde AVANT la conversion numpy pour garder les métadonnées si besoin
# Mais sans les colonnes inutiles
print("Sauvegarde des fichiers Parquet...")

# On garde DateTransaction dans le parquet au cas où on voudrait refaire des analyses
# On l'enlèvera juste au moment du fit() dans le notebook suivant.
train_df.write_parquet("data/processed/train.parquet")
test_df.write_parquet("data/processed/test.parquet")

print("✅ Sauvegarde terminée dans 'data/processed/'")

Sauvegarde des fichiers Parquet...
✅ Sauvegarde terminée dans 'data/processed/'
